In [ ]:
import requests
import time
import pandas as pd

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
url = "https://ows.goszakup.gov.kz/v3/subject/all"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  # Загружаем 500 записей за раз
all_data = []  
search_after = None  # Начальное значение для пагинации

while True:
    try:
        if search_after:  
            params.update({"page": "next", "search_after": search_after})

        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Загрузка завершена, новых данных нет.")
            break

        all_data.extend(items)
        print(f"Загружено {len(all_data)} записей...")

        # Обновляем `search_after` для следующей страницы
        search_after = items[-1]["pid"]  # Берем ID последней записи в текущем ответе

        # Промежуточное сохранение каждые 5000 записей
        if len(all_data) % 1000 < 500:
            pd.DataFrame(all_data).to_csv(
                "goszakup_subjAll_data.csv",
                index=False,
                encoding="utf-8-sig",
                sep=";",  # Меняем разделитель, если запятая конфликтует
                quoting=1,  # csv.QUOTE_NONNUMERIC (обернуть строки в кавычки)
                quotechar='"',
                escapechar="\\"
            )
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
df = pd.DataFrame(all_data)
df.to_csv(
    "goszakup_subjAll_data.csv",
    index=False,
    encoding="utf-8-sig",
    sep=";",  # Меняем разделитель, тк запятая конфликтует
    quoting=1,  # csv.QUOTE_NONNUMERIC (обернуть строки в кавычки)
    quotechar='"',
    escapechar="\\"
)

print("Данные сохранены в goszakup_subjAll_data.csv")